## Purpose

This script is testing out the pyspatial library as a tool to create a random forest model on raster data and use that model to predict. 

## Read in raster and calculate spectral indices

1. Read in and rename raw raster using pyspatialml Raster function
2. Calculate spectral indicies
3. Use stack function to create new raster of indices

In [ ]:
from pyspatialml import Raster

raw_fp = "data_raw/" # file path to raw data folder
raster_fp = raw_fp + 'white_small_sub.tif' # combine with tif file
stack_raw = Raster(raster_fp)
stack_raw.names
stack_raw.rename({'white_small_sub_1': 'coastal'})
stack_raw.rename({'white_small_sub_2': 'blue'})
stack_raw.rename({'white_small_sub_3': 'green'})
stack_raw.rename({'white_small_sub_4': 'yellow'})
stack_raw.rename({'white_small_sub_5': 'red'})
stack_raw.rename({'white_small_sub_6': 'rededge'})
stack_raw.rename({'white_small_sub_7': 'NIR1'})
stack_raw.rename({'white_small_sub_8': 'NIR2'})

In [ ]:
# Define nre_func
def nre_fun(x, y):
    nre = (x - y) / (x + y)
    return nre


In [ ]:
# Warning takes ~10 min to calc all indicies below
green_red = nre_fun(stack_raw['green'], stack_raw['red'])
blue_coastal = nre_fun(stack_raw['blue'], stack_raw['coastal'])
NIR2_yellow = nre_fun(stack_raw['NIR2'], stack_raw['yellow'])
NIR1_red = nre_fun(stack_raw['NIR1'], stack_raw['red'])
rededge_yellow = nre_fun(stack_raw['rededge'], stack_raw['yellow'])
red_NIR2 = nre_fun(stack_raw['red'], stack_raw['NIR2'])
rededge_NIR2 = nre_fun(stack_raw['rededge'], stack_raw['NIR2'])
rededge_NIR1 = nre_fun(stack_raw['rededge'], stack_raw['NIR1'])
green_NIR1 = nre_fun(stack_raw['green'], stack_raw['NIR1'])
green_NIR2 = nre_fun(stack_raw['green'], stack_raw['NIR2'])
rededge_green = nre_fun(stack_raw['rededge'], stack_raw['green'])
rededge_red = nre_fun(stack_raw['rededge'], stack_raw['red'])
yellow_NIR1 = nre_fun(stack_raw['yellow'], stack_raw['NIR1'])
NIR2_blue = nre_fun(stack_raw['NIR2'], stack_raw['blue'])
blue_red = nre_fun(stack_raw['blue'], stack_raw['red'])

# Make list of all indicies
predictors = [green_red, blue_coastal, NIR2_yellow, NIR1_red,
              rededge_yellow, red_NIR2, rededge_NIR2,
              rededge_NIR1, green_NIR1, green_NIR2, rededge_green,
              rededge_red, yellow_NIR1, NIR2_blue, blue_red]

# Stack indicies in new raster
stack = Raster(predictors)

# Rename each raster in stack
feature_names_orig = stack.names

feature_names = ['green red', 'blue coastal', 'NIR2 yellow', 'NIR1 red',
              'rededge yellow', 'red NIR2', 'rededge NIR2', 'rededge NIR1',
              'green NIR1', 'green NIR2', 'rededge green', 'rededge red',
              'yellow NIR1', 'NIR2 blue', 'blue red']


for n in range(len(feature_names_orig)):
    stack.rename({feature_names_orig[n]:feature_names[n]})

## Read in shapefile and extract raster values

Attempted to use extract_vector with all polygons. Script ran all night and did not finish. Maybe faster to use rasterio mask()?

Trying with a small subsample of polygons.

In [ ]:
import geopandas as gpd
import pandas as pd

shapefile_fp = raw_fp + 'Lindsay_white_river_land_cover/Lindsay_white_river_land_cover.shp'
training_poly = gpd.read_file(shapefile_fp)


training_poly.Classname = pd.Categorical(training_poly.Classname)
training_poly.Classcode = training_poly.Classname.cat.codes


# Create a small testing shapefile --- !!!full run will need to delete this!!!
training_poly = training_poly.groupby('Classcode').head(1)

The extraction step below takes a long time using the pyspatialml `extract_vector` tool...

In [ ]:
# https://pyspatialml.readthedocs.io/en/latest/mlworkflow.html#extraction-training-data
# had to install rasterio v1.1.5
# https://github.com/stevenpawley/Pyspatialml/issues/17
df_polygons = stack.extract_vector(training_poly)


In [ ]:
# Add Classvalue column to dataframe
df_polygons = df_polygons.merge(training_poly.loc[:, ("Classcode", "Classname")], right_index=True, left_on = 'id')

## Train RF model

First step is to sample from each class to get even distribution of classes.

NOTE: will need to remove the replace for the full model. Wood does not have enough samples here (n = 288)

In [ ]:
# Undersample the extracted data
nsamples_class = 5000 # Number of samples to take from each class
sample_seed = 42 # seed for random sample
features = df_polygons.groupby('Classcode').apply(lambda s: s.sample(nsamples_class, 
                                                            random_state = sample_seed, 
                                                            replace=True))


Next step is to clean up any rows with NA values and select the Classvalue as the y predictor

In [ ]:
# X data for rf
features = features.dropna()

# y data for rf
labels = features['Classcode']

In [ ]:
from sklearn.model_selection import train_test_split

# Partition data into testing and training data
X_train, X_test, y_train, y_test = train_test_split(features[feature_names],
                                                    labels, train_size = 0.9,
                                                    random_state = 42,
                                                    stratify = labels)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators = 200,
                            max_features = 5,
                            random_state = 8)

rf.fit(X_train, y_train)

### Test accuracy of model

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.inspection import permutation_importance


result = permutation_importance(rf, X_train, y_train, random_state = 8)
predictions = rf.predict(X_test)
accuracy = accuracy_score(y_test, predictions)

accuracy

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix

confmat = confusion_matrix(y_test, predictions)
df_confmat = pd.DataFrame(confmat)
plot_confusion_matrix(rf, X_test, y_test)

## Use RF model to predict raster stack

So far have only tested with a dummy model, but seems pretty fast and easy to implement. 

In [ ]:
# Predictions on full stack
result = stack.predict(estimator=rf, dtype = 'int16', nodata=0)


In [ ]:
# plot classification result
import matplotlib.pyplot as plt

result.iloc[0].cmap = "Dark2"
result.iloc[0].categorical = True
result.plot()
plt.show()

## Testing below here